## Bank International Settlement Pipeline

### Dave Friesen & Lane Whitmore

In [1]:
### imports
# test update
import numpy as np
import pandas as pd
import pymysql as mysql

### CSV Imports to pd Dataframe

In [2]:
policy_rates = pd.read_csv("C:/Users/whitm/OneDrive/Desktop/ADS507/final/WS_CBPOL_M_csv_col.csv")
exchange_rates = pd.read_csv("C:/Users/whitm/OneDrive/Desktop/ADS507/final/WS_XRU_csv_col.csv")
consumer_prices = pd.read_csv("C:/Users/whitm/OneDrive/Desktop/ADS507/final/WS_LONG_CPI_csv_col.csv")

In [3]:
policy_rates.head()

,FREQ,Frequency,REF_AREA,Reference area,TIME_FORMAT,Time Format,COMPILATION,DECIMALS,Decimals,SOURCE_REF,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,M,Monthly,AR,Argentina,NaN,NaN,From 08 Aug 2018: 7 Day Liquidity Bills (LELIQ...,4,Four,Central Bank of Argentina,...,44.50,47.00,49.00,52.00,60.00,69.50,75.00,75.00,75.00,75.00
1,M,Monthly,AU,Australia,NaN,NaN,From 2 Aug 1990 onwards: cash rate target; fro...,4,Four,Reserve Bank of Australia,...,0.10,0.10,0.35,0.85,1.35,1.85,2.35,2.60,2.85,3.10
2,M,Monthly,BR,Brazil,NaN,NaN,"From 5 Mar 1999 onwards Central Bank target, m...",4,Four,Central Bank of Brazil,...,11.75,11.75,12.75,13.25,13.25,13.75,13.75,13.75,13.75,13.75
3,M,Monthly,CA,Canada,NaN,NaN,"From 1 Jun 1994 onwards: Central Bank target, ...",4,Four,Bank of Canada,...,0.50,1.00,1.00,1.50,2.50,2.50,3.25,3.75,3.75,4.25
4,M,Monthly,CH,Switzerland,NaN,NaN,From 13 June 2019 onwards SNB Policy rate; Fro...,4,Four,Swiss National Bank,...,-0.75,-0.75,-0.75,-0.25,-0.25,-0.25,0.50,0.50,0.50,1.00


In [4]:
exchange_rates.head()

,FREQ,Frequency,REF_AREA,Reference area,CURRENCY,Currency,COLLECTION,Collection,UNIT_MULT,Unit Multiplier,...,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2022-Q1,2022-Q2,2022-Q3,2022-Q4
0,A,Annual,AE,United Arab Emirates,AED,UAE dirham,A,Average of observations through period,0,Units,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Annual,AE,United Arab Emirates,AED,UAE dirham,E,End of period,0,Units,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Annual,AF,Afghanistan,AFN,Afghani,A,Average of observations through period,0,Units,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Annual,AF,Afghanistan,AFN,Afghani,E,End of period,0,Units,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Annual,AG,Antigua and Barbuda,XCD,Eastern Caribbean dollar,A,Average of observations through period,0,Units,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
consumer_prices.head()

,FREQ,Frequency,REF_AREA,Reference area,UNIT_MEASURE,Unit of measure,TIME_FORMAT,Time Format,BREAKS,COVERAGE,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,A,Annual,AE,United Arab Emirates,628,"Index, 2010 = 100",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Annual,AE,United Arab Emirates,771,"Year-on-year changes, in per cent",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Annual,AR,Argentina,628,"Index, 2010 = 100",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Annual,AR,Argentina,771,"Year-on-year changes, in per cent",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Annual,AT,Austria,628,"Index, 2010 = 100",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Pandas "Melt" to transpose dataframe from wide to long 

In [45]:
pr_df = pd.melt(policy_rates, id_vars = policy_rates.iloc[:, 0:12], value_vars = policy_rates.iloc[:,13:937])
pr_df.head()

,FREQ,Frequency,REF_AREA,Reference area,TIME_FORMAT,Time Format,COMPILATION,DECIMALS,Decimals,SOURCE_REF,SUPP_INFO_BREAKS,TITLE,variable,value
0,M,Monthly,AR,Argentina,NaN,NaN,From 08 Aug 2018: 7 Day Liquidity Bills (LELIQ...,4,Four,Central Bank of Argentina,This rate can be considered the official polic...,Central bank policy rates - Argentina - Month...,1946-01,NaN
1,M,Monthly,AU,Australia,NaN,NaN,From 2 Aug 1990 onwards: cash rate target; fro...,4,Four,Reserve Bank of Australia,This rate can be considered the official polic...,Central bank policy rates - Australia - Month...,1946-01,NaN
2,M,Monthly,BR,Brazil,NaN,NaN,"From 5 Mar 1999 onwards Central Bank target, m...",4,Four,Central Bank of Brazil,This rate can be considered the official polic...,Central bank policy rates - Brazil - Monthly ...,1946-01,NaN
3,M,Monthly,CA,Canada,NaN,NaN,"From 1 Jun 1994 onwards: Central Bank target, ...",4,Four,Bank of Canada,In Jun 1994 the Bank of Canada (BoC) began shi...,Central bank policy rates - Canada - Monthly ...,1946-01,NaN
4,M,Monthly,CH,Switzerland,NaN,NaN,From 13 June 2019 onwards SNB Policy rate; Fro...,4,Four,Swiss National Bank,NaN,Central bank policy rates - Switzerland - Mon...,1946-01,1.5


In [46]:
cp_df = pd.melt(consumer_prices, id_vars = consumer_prices.iloc[:, 0:13], value_vars = consumer_prices.iloc[:, 14:1696])
cp_df

,FREQ,Frequency,REF_AREA,Reference area,UNIT_MEASURE,Unit of measure,TIME_FORMAT,Time Format,BREAKS,COVERAGE,DECIMALS,Decimals,TITLE_TS,variable,value
0,A,Annual,AE,United Arab Emirates,628,"Index, 2010 = 100",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1661,NaN
1,A,Annual,AE,United Arab Emirates,771,"Year-on-year changes, in per cent",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1661,NaN
2,A,Annual,AR,Argentina,628,"Index, 2010 = 100",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1661,NaN
3,A,Annual,AR,Argentina,771,"Year-on-year changes, in per cent",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1661,NaN
4,A,Annual,AT,Austria,628,"Index, 2010 = 100",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1661,NaN


In [49]:
exr_df = pd.melt(exchange_rates, id_vars = exchange_rates.iloc[:, 0:15], value_vars = exchange_rates.iloc[:, 16:3960])
exr_df.head()

,FREQ,Frequency,REF_AREA,Reference area,CURRENCY,Currency,COLLECTION,Collection,UNIT_MULT,Unit Multiplier,DECIMALS,Decimals,AVAILABILITY,Availability,TITLE,variable,value
0,A,Annual,AE,United Arab Emirates,AED,UAE dirham,A,Average of observations through period,0,Units,6,Six,A,All users,Exchange rates against USD United Arab Emirat...,1791,NaN
1,A,Annual,AE,United Arab Emirates,AED,UAE dirham,E,End of period,0,Units,6,Six,A,All users,Exchange rates against USD United Arab Emirat...,1791,NaN
2,A,Annual,AF,Afghanistan,AFN,Afghani,A,Average of observations through period,0,Units,6,Six,A,All users,Exchange rates against USD Afghanistan - Afgh...,1791,NaN
3,A,Annual,AF,Afghanistan,AFN,Afghani,E,End of period,0,Units,6,Six,A,All users,Exchange rates against USD Afghanistan - Afgh...,1791,NaN
4,A,Annual,AG,Antigua and Barbuda,XCD,Eastern Caribbean dollar,A,Average of observations through period,0,Units,6,Six,A,All users,Exchange rates against USD Antigua and Barbud...,1791,NaN
